In [ ]:
# hair removal- commonly done in alot of peoples approaches however I cant see it within Gesserts work 



In [ ]:
# cropping out black border unfortunately Gessert uses Matlab for doing this https://github.com/ngessert/isic2019/blob/2394aaf60e7e070a5d0197a381426a1259a5f1bb/Matlab/adjust_2019.m 